### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import timm
from model import LatenViTtiny
import os
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from PIL import Image
from collections import OrderedDict

c:\Users\Haseeb\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration

In [14]:

class Config:
    MODEL_NAME   = 'tiny_vit_21m_224.dist_in22k_ft_in1k'
    NUM_CLASSES  = 7      
    NREPEAT      = 2
    stage = 2
    
    BATCH_SIZE   = 128
    NUM_EPOCHS   = 5
    LEARNING_RATE= 1e-4
    DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    DATA_ROOT    = "../../../pacs_data/pacs_data"
    DOMAINS      = ["art_painting", "cartoon", "photo", "sketch"]
    
    TRANSFORM = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])


### PACS Dataset Class

In [3]:
class PACSDataset(Dataset):
    def __init__(self, root_dir, domain, transform=None):
        self.root_dir    = os.path.join(root_dir, domain)
        self.transform   = transform
        self.classes     = sorted(os.listdir(self.root_dir))
        self.class_to_idx= {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images      = []
        self.labels      = []
        
        for cls_name in self.classes:
            cls_dir = os.path.join(self.root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append(os.path.join(cls_dir, img_name))
                self.labels.append(self.class_to_idx[cls_name])
                
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image    = Image.open(img_path).convert('RGB')
        label    = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        return image, label


### Model Setup

In [4]:
def setup_model():
    base_model = timm.create_model(Config.MODEL_NAME, pretrained=True)
    model = LatenViTtiny(
        model     = base_model,
        nrepeat   = Config.NREPEAT,
        stage = Config.stage
    )
    return model.to(Config.DEVICE)

def setup_baseline_model():
    base_model = timm.create_model(Config.MODEL_NAME, pretrained=True)
    base_model.head = nn.Linear(base_model.head.in_features, Config.NUM_CLASSES)
    return base_model.to(Config.DEVICE)


### Training Function

In [5]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct      = 0
    total        = 0
    
    for images, labels in dataloader:
        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted  = outputs.max(1)
        total        += labels.size(0)
        correct      += predicted.eq(labels).sum().item()
        
    epoch_loss = running_loss / len(dataloader)
    epoch_acc  = 100.0 * correct / total
    return epoch_loss, epoch_acc


### Training Function

In [6]:
@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total   = 0
    
    for images, labels in dataloader:
        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
        outputs        = model(images)
        
        _, predicted = outputs.max(1)
        total       += labels.size(0)
        correct     += predicted.eq(labels).sum().item()
        
    return 100.0 * correct / total


### Baseline -CotFormer

In [15]:

train_loaders = []
test_loaders  = []
for domain in Config.DOMAINS:
    ds_train = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    ds_test  = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))
    test_loaders .append(DataLoader(ds_test,  batch_size=Config.BATCH_SIZE, shuffle=False))


full_train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
full_test_ds  = ConcatDataset([dl.dataset  for dl in test_loaders ])
full_train_loader = DataLoader(full_train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)
full_test_loader  = DataLoader(full_test_ds,  batch_size=Config.BATCH_SIZE, shuffle=False)

model     = setup_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

for epoch in range(1, Config.NUM_EPOCHS + 1):
    loss, acc = train_epoch(model, full_train_loader, criterion, optimizer)
    print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

test_acc = evaluate(model, full_test_loader)
print(f"Baseline (all domains) Test Accuracy: {test_acc:.2f}%")

print("\n[Per-Domain Evaluation]")
domain_accuracies = OrderedDict()
for domain, loader in zip(Config.DOMAINS, test_loaders):
    acc = evaluate(model, loader)
    domain_accuracies[domain] = acc
    print(f"  {domain:>12}: {acc:.2f}%")

[Epoch 1/5] Train Loss: 3.7481, Train Acc: 14.11%


KeyboardInterrupt: 

In [ ]:

transform = Config.TRANSFORM  

lodo_results = OrderedDict()

for test_domain in Config.DOMAINS:
    print(f"\n=== LODO: Held-Out Domain = {test_domain} ===")
    train_loaders = []
    for d in Config.DOMAINS:
        if d == test_domain:
            continue
        ds_train = PACSDataset(Config.DATA_ROOT, d, transform)
        train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))

    train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
    train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)

    ds_test    = PACSDataset(Config.DATA_ROOT, test_domain, transform)
    test_loader = DataLoader(ds_test, batch_size=Config.BATCH_SIZE, shuffle=False)

    model     = setup_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

    for epoch in range(1, Config.NUM_EPOCHS + 1):
        loss, acc = train_epoch(model, train_loader, criterion, optimizer)
        print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

    test_acc = evaluate(model, test_loader)
    lodo_results[test_domain] = test_acc
    print(f"--> Test Accuracy on {test_domain}: {test_acc:.2f}%")

print("\n=== LODO Summary ===")
for domain, acc in lodo_results.items():
    print(f"{domain:>14}: {acc:.2f}%")


### Baseline Vanilla

In [ ]:

train_loaders = []
test_loaders  = []
for domain in Config.DOMAINS:
    ds_train = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    ds_test  = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))
    test_loaders .append(DataLoader(ds_test,  batch_size=Config.BATCH_SIZE, shuffle=False))


full_train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
full_test_ds  = ConcatDataset([dl.dataset  for dl in test_loaders ])
full_train_loader = DataLoader(full_train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)
full_test_loader  = DataLoader(full_test_ds,  batch_size=Config.BATCH_SIZE, shuffle=False)

model     = setup_baseline_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

for epoch in range(1, Config.NUM_EPOCHS + 1):
    loss, acc = train_epoch(model, full_train_loader, criterion, optimizer)
    print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

test_acc = evaluate(model, full_test_loader)
print(f"Baseline (all domains) Test Accuracy: {test_acc:.2f}%")

print("\n[Per-Domain Evaluation]")
domain_accuracies = OrderedDict()
for domain, loader in zip(Config.DOMAINS, test_loaders):
    acc = evaluate(model, loader)
    domain_accuracies[domain] = acc
    print(f"  {domain:>12}: {acc:.2f}%")

In [ ]:

transform = Config.TRANSFORM  

lodo_results = OrderedDict()

for test_domain in Config.DOMAINS:
    print(f"\n=== LODO: Held-Out Domain = {test_domain} ===")
    train_loaders = []
    for d in Config.DOMAINS:
        if d == test_domain:
            continue
        ds_train = PACSDataset(Config.DATA_ROOT, d, transform)
        train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))

    train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
    train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)

    ds_test    = PACSDataset(Config.DATA_ROOT, test_domain, transform)
    test_loader = DataLoader(ds_test, batch_size=Config.BATCH_SIZE, shuffle=False)

    model     = setup_baseline_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

    for epoch in range(1, Config.NUM_EPOCHS + 1):
        loss, acc = train_epoch(model, train_loader, criterion, optimizer)
        print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

    test_acc = evaluate(model, test_loader)
    lodo_results[test_domain] = test_acc
    print(f"--> Test Accuracy on {test_domain}: {test_acc:.2f}%")

print("\n=== LODO Summary ===")
for domain, acc in lodo_results.items():
    print(f"{domain:>14}: {acc:.2f}%")
